In [13]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA


df1 = pd.read_csv('data/stock_details_5_years.csv', header=0)

def time_fixer(x):
    x = x.split(' ')[0]
    return x

df1['Date'] = df1['Date'].apply(time_fixer)
df1['Date'] = pd.to_datetime(df1['Date'])

df1.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Company
0,2018-11-29,43.829761,43.863354,42.639594,43.083508,167080000,0.00,0.0,AAPL
1,2018-11-29,104.769074,105.519257,103.534595,104.636131,28123200,0.00,0.0,MSFT
2,2018-11-29,54.176498,55.007500,54.099998,54.729000,31004000,0.00,0.0,GOOGL
3,2018-11-29,83.749496,84.499496,82.616501,83.678497,132264000,0.00,0.0,AMZN
4,2018-11-29,39.692784,40.064904,38.735195,39.037853,54917200,0.04,0.0,NVDA


In [14]:
#todo - add comments explaining code later
company_opens = df1.groupby('Company')['Open'].apply(np.array)
company_closes = df1.groupby('Company')['Close'].apply(np.array)

changes_df = pd.DataFrame(index=company_opens.index)
changes_df['opens'] = company_opens
changes_df['closes'] = company_closes

def perc_change(open_ser, close_ser):

    return ((close_ser - open_ser) / open_ser )*100.0
    
data1 = perc_change(changes_df['opens'], changes_df['closes'] )

changes_df['perc_change'] = data1

full_length = changes_df['perc_change'].apply(len).max()

full_companies = changes_df['perc_change'].apply(len)

useable_companies = full_companies[full_companies == full_length]

# useable_companies

final_df = changes_df.loc[useable_companies.index]

final_df = final_df.drop(columns=['opens', 'closes'])
final_df.head(10)

,perc_change
Company,
A,"[0.4773218052326546, 1.0051671761984158, 1.827..."
AAPL,"[-1.702617183838854, -0.9484672021502785, 0.19..."
ABBV,"[1.011124035815082, 4.442714837058878, -0.6997..."
ABEV,"[0.7025810290951009, 0.2347359508779906, -0.68..."
ABT,"[1.1459365284516383, 0.1623194184859994, -0.02..."
ACGL,"[0.38910046417311844, 0.5975397952022214, -1.1..."
ACN,"[-0.702792270799572, 1.6999443205598022, 0.512..."
ADBE,"[1.1081327004284363, 0.2637588112448539, -2.09..."
ADI,"[-0.5112321590414718, 2.4749178330342025, 0.18..."
